<a href="https://colab.research.google.com/github/Erika-Hettiaratchy/API-EmailValidation/blob/main/Scraping_Yahoo_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance gspread oauth2client pandas


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving scraping-from-yahoo-finance-5c54fbc2d784.json to scraping-from-yahoo-finance-5c54fbc2d784.json
Saving scraping-from-yahoo-finance-f5468746e38f.json to scraping-from-yahoo-finance-f5468746e38f.json


In [ ]:
creds = ServiceAccountCredentials.from_json_keyfile_name("scraping-from-yahoo-finance-f5468746e38f.json", scope)


In [ ]:

import yfinance as yf
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime

# === Google Sheets Authentication ===
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("scraping-from-yahoo-finance-5c54fbc2d784.json", scope)
client = gspread.authorize(creds)

# === Fetch Historical Stock Prices from Yahoo Finance ===
tickers = ["AAPL", "GOOGL", "MSFT", "TSLA"]  # Add more ticker symbols as needed
start_date = "2023-01-20"  # Start tracking from this date # Changed date to retrieve actual data
end_date = datetime.date.today().strftime("%Y-%m-%d")  # Today's date

# Create a DataFrame to store all stock data
df = pd.DataFrame()

# Fetch stock history for each ticker
for ticker in tickers:
    stock = yf.Ticker(ticker)
    history = stock.history(start=start_date, end=end_date)[["Close"]]  # Get closing prices
    history.columns = [ticker]  # Rename column with ticker symbol
    df = pd.concat([df, history], axis=1)  # Merge all stocks into one DataFrame

# Reset index to move Date from index to a column
df.reset_index(inplace=True)
df.rename(columns={"Date": "Date"}, inplace=True)

# === Convert Timestamp objects to strings before updating ===
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d') # Convert 'Date' column to strings

# === Open Google Sheet and Update ===
sheet = client.open("Stock Prices").sheet1  # Ensure you have a sheet named "Stock Prices"
sheet.clear()  # Clear old data before updating
sheet.update([df.columns.tolist()] + df.values.tolist())  # Update with new data

print("Historical stock data updated in Google Sheets!")

Historical stock data updated in Google Sheets!


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
import yfinance as yf

# Google Sheets authentication
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("scraping-from-yahoo-finance-f5468746e38f.json", scope)
client = gspread.authorize(creds)

# Open Google Sheet
sheet = client.open("Stock Prices").sheet1  # Replace with your actual sheet name

# Define stock tickers
tickers = ["AAPL", "GOOGL", "TSLA"]  # Add any stocks you want to track

# Get today's date
today = datetime.date.today().strftime("%Y-%m-%d")

# Fetch stock prices
data = []
for ticker in tickers:
    stock = yf.Ticker(ticker)
    price = stock.history(period="1d")["Close"].iloc[-1]
    data.append([today, ticker, price])

# Convert to DataFrame
df = pd.DataFrame(data, columns=["Date", "Ticker", "Price"])

# Get existing data to check if today's date is already recorded
existing_data = sheet.get_all_values()
if existing_data:
    last_date = existing_data[-1][0]  # Get last recorded date
    if last_date == today:
        print("Today's stock prices are already recorded.")
    else:
        sheet.append_rows(df.values.tolist())  # Append new data
        print("Stock data appended successfully!")
else:
    sheet.update("A1", [["Date", "Ticker", "Price"]] + df.values.tolist())
    print("Stock data added successfully!")


Stock data appended successfully!
